In [ ]:
#!/bin/bash

# Arithmetic Operations

sum=3

echo $((sum**10))   # using (())
expr $sum \* 9      # using expr  
echo $sum ^ 19 | bc # using bc


In [ ]:
#!/bin/bash

# Boolean Operations

true ; echo $?
false ; echo $?

[[ 1 -eq 2 ]] ; echo $?
[[ 1 -le 2 ]] ; echo $? # -le -gt -ge -

In [ ]:
#!/bin/bash
# FOR LOOP
for integer in 1 2 3 4 5 6 7 
    do 
        echo $integer
    done

In [ ]:
#!/bin/bash

for word in some string statement
    do
        echo $word
    done

In [ ]:
#!/bin/bash

for i in $(ls /etc/); 
    do
        echo $i 
    done

In [ ]:
#!/bin/bash

for ((i=0; i<=10; i++))
    do
        echo $i
    done


In [ ]:
#!/bin/bash

# WHILE LOOP

while <test-command>
    do
        <some commands..>
    done

In [ ]:
#!/bin/bash

# WHILE LOOP

i=0
while [ $i -le 10 ] 
    do
        echo $i
        ((i++))
    done

In [ ]:
#!/bin/bash

# UNTIL LOOP

i=0
until test $i -eq 10 
    do
        echo $i
        ((i++))
    done

In [ ]:
#!/bin/bash

# CONDITIONAL EXPRESSIONS
# IF FI
if [ "foo" = "foo" ]; then
    echo expression evaluated as true
fi

In [ ]:
#!/bin/bash

# check if a number is even

read -p "Enter a number: " i

if (( i % 2 == 0 )); then
    echo "$i is even"
fi

In [ ]:
#!/bin/bash

# IF ELSE

# check if a number is even

read -p "Enter a number: " i


if ! [[ "$i" =~ ^[0-9]+$ ]]; then  # check if the input is an integer
    echo "Error: Not an integer"
    exit 1
else
    if (( i % 2 == 0 )); then # check if even or odd
        echo "$i is even"
    else 
        echo "$i is odd"
    fi
fi

In [ ]:
#!/bin/bash

# IF ELSE
# FUNCTION
# WHILE LOOP

# check if a number is integer first then check if it is even

read -p "Enter a number: " i

# check if the input is an integer
function is_int(){
    if [[ "$i" =~ ^[0-9]+$ ]]; then
        return 0
    else
        return 1
    fi
}

if is_int ; then  # check if the input is an integer
    if (( i % 2 == 0 )); then # check if even or odd
        echo "$i is even"
    else 
        echo "$i is odd"
    fi
else
    echo "Error: Not an integer"
fi

In [ ]:
#!/bin/bash

# IF ELSE
# FUNCTIONS
# WHILE
# keep checking if a number is integer and then if it's even until a value is entered



function is_int(){
    if [[ "$i" =~ ^[0-9]+$ ]]; then
        return 0
    else
        return 1
    fi
}

while [[ ${i,,} != 'khalas' ]]
do

read -p "Enter a number: " i

if is_int ; then  # check if the input is an integer
    if (( i % 2 == 0 )); then # check if even or odd
        echo "$i is even"
    else 
        echo "$i is odd"
    fi
else
    echo "Error: Not an integer"
fi

done

In [ ]:
#!/bin/bash

# Check if Docker is installed
if ! command -v docker &> /dev/null; then
    echo "Docker is not installed. Installing..."
    # check if curl is installed
    if ! command -v curl &> /dev/null; then
        sudo apt install curl -y
    fi
    curl -fsSL https://get.docker.com -o get-docker.sh
    sudo sh get-docker.sh
else
    echo "Docker is already installed."
fi

# Check if Docker service is enabled
if ! systemctl is-active --quiet docker; then
    echo "Enabling Docker service..."
    sudo systemctl start docker
    sudo systemctl enable docker
else
    echo "Docker service is already enabled."
fi

echo "Docker setup complete!"


In [ ]:
#!/bin/bash

# kill any mysql active containers (if any)
container_id=$(sudo docker container ls --filter "ancestor=mysql" -q)
if ! [[ -z $container_id ]]; then
    echo "Existing running mysql container found and being deleted.."
    sudo docker rm $container_id -f
fi 

# pull and run the mysql container
sudo docker run --rm --name mysql-db -h mysql-db -e MYSQL_ROOT_PASSWORD=P@ssw0rd -d mysql:latest && echo "mysql container is running.."

for ((attempt = 1; attempt <= 9; attempt++)); do
    if sudo docker exec -it mysql-db mysql -uroot -pP@ssw0rd -e "SELECT 1;" &> /dev/null; then
        echo "Connected to successfully to mysql database engine"
        break
    else
        echo "Connection failed to mysql engine, attempting reconnection $attempt/9 ... "
        
        sleep 5
    fi
done

# Create mysql database and table 
sudo docker exec -i mysql-db mysql -w --show-warnings=0 -uroot -pP@ssw0rd -e 'CREATE DATABASE namesdb; USE namesdb; CREATE TABLE passwd (name VARCHAR(20),uid INT,gid INT,primary_group VARCHAR(30),home_dir VARCHAR(30),default_shell VARCHAR(30));' 2> /dev/null

# Read /etc/passwd and extract user information
# Use the cut command to extract the corrsponding field and create arrays
_usernames=$(cut -d: -f1 /etc/passwd)
_uids=$(cut -d: -f3 /etc/passwd)
_gids=$(cut -d: -f4 /etc/passwd)
_primary_groups=$(cut -d: -f5 /etc/passwd)
_home_dirs=$(cut -d: -f6 /etc/passwd)
_default_shells=$(cut -d: -f7 /etc/passwd)

# We'll need to rebuild the arrays as the cut command doesn't put it in a proper format
declare -a usernames uids gids primary_groups home_dirs default_shells

n=0
for name in $_usernames
do
    usernames[n]=$name
    ((n++))
done

n=0
for uid in $_uids
do
    uids[n]=$uid
    ((n++))
done

n=0
for gid in $_gids
do
    gids[n]=$gid
    ((n++))
done

n=0
for pg in $_primary_groups
do
    primary_groups[n]=$pg
    ((n++))
done

n=0
for hd in $_home_dirs
do
    home_dirs[n]=$hd
    ((n++))
done

n=0
for ds in $_default_shells
do
    default_shells[n]=$ds
    ((n++))
done

# Validation of arrays
# for ((i=0;i<$n;i++))
# do
#     echo ${usernames[i]} ${default_shells[i]}
# done

for ((i=0;i<$n;i++))
do
    sudo docker exec mysql-db mysql -w --show-warnings=0  -uroot -pP@ssw0rd -e "USE namesdb; INSERT INTO passwd VALUES('${usernames[i]}', '${uids[i]}', '${gids[i]}', '${primary_groups[i]}', '${home_dirs[i]}', '${default_shells[i]}')" 2> /dev/null
done

echo "Data loaded successfully!"
